# PRE-PROCESSING (NEW DATA)
In this notebook we pre-process the new dataset containing all the features extracted in the *Feature Extraction* file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
import joblib

In [ ]:
# Set Pandas to display all columns
pd.set_option('display.max_columns', None)

In [ ]:
def load_tsv_as_df(folder, filename):
    if not filename.endswith('.tsv'):
        filename += '.tsv'
    path = os.path.join(folder, filename)
    df = pd.read_csv(path, sep='\t')
    return df

In [ ]:
def save_df_as_tsv(df, folder, filename):
    os.makedirs(folder, exist_ok=True)
    if not filename.endswith('.tsv'):
        filename += '.tsv'
    path = os.path.join(folder, filename)
    df.to_csv(path, sep='\t', index=False)
    print(f"Saved TSV to: {path}")

# Data Loading
The tsv file is loaded directly into a pandas dataframe.

In [ ]:
# Specify your folder path
folder_path = '/content/drive/MyDrive'

# Path to the TSV file
tsv_path = f'{folder_path}/Structural Bioinfo PROJECT/datasets'

# Load df
combined_df = load_tsv_as_df(tsv_path, 'combinated_df_new.tsv')

In [ ]:
# Show the result
print("Shape of the dataset:", combined_df.shape)
combined_df.head()

Shape of the dataset: (2968986, 44)


,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,s_3di_state,s_3di_letter,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,t_3di_state,t_3di_letter,Interaction,same_chain,delta_rsa,delta_atchley_1,delta_atchley_2,delta_atchley_3,delta_atchley_4,delta_atchley_5,ca_distance,s_centroid_x,s_centroid_y,t_centroid_x,t_centroid_y
0,1b0y,A,28,,R,H,0.056,-1.127,-0.711,1.538,-0.055,1.502,0.440,2.897,13.0,N,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,VDW,1,0.378,2.129,1.247,2.235,1.130,3.043,4.967376,-1.106118,-1.339661,-1.140001,-2.006822
1,1b0y,A,27,,E,S,0.531,-1.622,0.483,1.357,-1.453,1.477,0.113,-0.837,1.0,B,A,31,,A,H,0.434,-1.221,-0.526,-0.591,-1.302,-0.733,1.570,-0.146,9.0,J,HBOND,1,0.097,1.948,0.151,2.210,1.457,0.691,6.062192,-0.135632,-1.891373,-1.140001,-2.006822
2,1b0y,A,47,,Q,T,0.460,-0.986,-0.566,0.931,-0.179,-3.005,-0.503,-1.853,13.0,N,A,84,,A,P,0.472,-1.557,2.544,-0.591,-1.302,-0.733,1.570,-0.146,2.0,C,HBOND,1,0.012,1.522,1.123,2.272,2.073,1.707,6.714707,-1.106118,-1.339661,0.494826,-0.420486
3,1b0y,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,A,45,,N,G,0.274,-1.941,0.412,0.945,0.828,1.299,-0.169,0.933,13.0,N,HBOND,1,0.272,0.412,2.281,0.178,0.282,1.770,8.626575,0.690114,-1.255422,-1.106118,-1.339661
4,1b0y,A,37,,P,-,0.551,-0.880,2.320,0.189,2.081,-1.628,0.421,-1.392,14.0,O,A,40,,E,G,0.546,-1.860,-0.062,1.357,-1.453,1.477,0.113,-0.837,12.0,M,HBOND,1,0.005,1.168,3.534,3.105,0.308,0.555,5.669582,2.149514,-0.802992,0.690114,-1.255422


We can immediately appreciate the added features/columns.

Assign the "MISSING" value to the interaction labels that are missing/unclassified.

In [ ]:
combined_df['Interaction'] = combined_df['Interaction'].fillna('MISSING')
combined_df['Interaction'] = combined_df['Interaction'].astype('category')
combined_df['Interaction'].cat.categories.tolist()

['HBOND',
 'IONIC',
 'MISSING',
 'PICATION',
 'PIHBOND',
 'PIPISTACK',
 'SSBOND',
 'VDW']

# Exploratory Data Analysis and Preprocessing

## Missing values

The number of missing counts per feature and the features with missing values are displayed.

In [ ]:
missing_counts = combined_df.isnull().sum()
print(missing_counts[missing_counts > 0])

s_ss8              31
s_rsa              63
s_phi           17807
s_psi            6736
s_3di_state     37025
s_3di_letter    37025
t_ss8              56
t_rsa              75
t_phi            6167
t_psi           21474
t_3di_state     44036
t_3di_letter    44036
delta_rsa         138
ca_distance     50115
s_centroid_x    37025
s_centroid_y    37025
t_centroid_x    44036
t_centroid_y    44036
dtype: int64


We can see that the missing 3di state variables propagate, expectedly, to the centroids columns. Finally, the CA distance also shows a relevant amount of missing values.

The same type of analysis we did on the original dataset is carried out on the new one:

In [ ]:
# Percentage of missing values per column, grouped by Interaction
missing_pct_by_interaction = combined_df.groupby('Interaction').apply(lambda g: g.isnull().mean() * 100).round(2)
print(missing_pct_by_interaction)

/tmp/ipython-input-10-2430425720.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  missing_pct_by_interaction = combined_df.groupby('Interaction').apply(lambda g: g.isnull().mean() * 100).round(2)


             pdb_id  s_ch  s_resi  s_ins  s_resn  s_ss8  s_rsa  s_phi  s_psi  \
Interaction                                                                    
HBOND           0.0   0.0     0.0    0.0     0.0    0.0   0.00   0.56   0.25   
IONIC           0.0   0.0     0.0    0.0     0.0    0.0   0.00   0.57   0.28   
MISSING         0.0   0.0     0.0    0.0     0.0    0.0   0.01   0.63   0.21   
PICATION        0.0   0.0     0.0    0.0     0.0    0.0   0.00   0.46   0.28   
PIHBOND         0.0   0.0     0.0    0.0     0.0    0.0   0.00   0.78   0.22   
PIPISTACK       0.0   0.0     0.0    0.0     0.0    0.0   0.00   0.40   0.30   
SSBOND          0.0   0.0     0.0    0.0     0.0    0.0   0.00   1.05   0.19   
VDW             0.0   0.0     0.0    0.0     0.0    0.0   0.00   0.63   0.22   

             s_a1  s_a2  s_a3  s_a4  s_a5  s_3di_state  s_3di_letter  t_ch  \
Interaction                                                                  
HBOND         0.0   0.0   0.0   0.0   0.0  

/tmp/ipython-input-10-2430425720.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  missing_pct_by_interaction = combined_df.groupby('Interaction').apply(lambda g: g.isnull().mean() * 100).round(2)


Similar conclusions to the ones we made for the original data can be drawn from the added features:
* Missing centroids propagate identically to 3di variables;
* Missing CA distance does not seem to be connected to the interaction type.

 Here too, as with the original data, we eliminate all incomplete observations.

In [ ]:
# Drop non-full observations
combined_df = combined_df.dropna()

In [ ]:
interaction_counts = combined_df['Interaction'].value_counts()
print(interaction_counts)

Interaction
MISSING      1038001
HBOND        1013182
VDW           706972
PIPISTACK      36836
IONIC          33841
PICATION        8425
SSBOND          2000
PIHBOND         1707
Name: count, dtype: int64


The resulting full dataset size is still satisfactory, despite the final number of rows being slightly smaller than with the original dataset, due to some additional observations lost with the CA distance.



In [ ]:
total_interactions = interaction_counts.sum()
print(f"Total number of interactions: {total_interactions}")

Total number of interactions: 2840964


## Features removal

Some features are removed immediately for computational efficieny:
* *3di_letter*  and *3di_state* are discarded for both source and target, since here we explore the alternative use of 3di centroid coordinates

Others are kept for the next steps and deleted afterwards:
* insertion code for both source and target residues are excluded as they don't carry much bio-physically meaningful information;
* PDB IDs;
* chain IDs which we replace with the *same-chain* variable.

In [ ]:
combined_df.drop(columns=['s_3di_letter', 't_3di_letter', 's_3di_state' ,'t_3di_state'], inplace=True)

Categorical variables are promptly encoded as so:

In [ ]:
categorical_cols = ['s_ch', 't_ch', 's_resn', 't_resn', 's_ins', 't_ins', 's_ss8', 't_ss8']

# Convert specified columns to 'category' dtype
combined_df[categorical_cols] = combined_df[categorical_cols].astype('category')

In [ ]:
combined_df.describe(include='all')

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,Interaction,same_chain,delta_rsa,delta_atchley_1,delta_atchley_2,delta_atchley_3,delta_atchley_4,delta_atchley_5,ca_distance,s_centroid_x,s_centroid_y,t_centroid_x,t_centroid_y
count,2840964,2840964,2.840964e+06,2840964,2840964,2840964,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2840964,2.840964e+06,2840964,2840964,2840964,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2840964,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06,2.840964e+06
unique,3883,33,NaN,12,20,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33,NaN,12,20,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,5ej8,A,NaN,,L,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,NaN,,L,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MISSING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,15539,1700310,NaN,2840744,261251,1183036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1628688,NaN,2840726,267782,1159176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1038001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2.084359e+02,NaN,NaN,NaN,1.706522e-01,-1.388896e+00,5.895814e-01,2.157048e-02,-9.003262e-02,-6.647259e-02,2.838041e-01,-5.991481e-02,NaN,2.432722e+02,NaN,NaN,NaN,1.821581e-01,-1.369148e+00,5.868840e-01,3.097279e-02,-1.299190e-01,2.861175e-02,2.755556e-01,4.168819e-02,NaN,9.597985e-01,1.599919e-01,1.021123e+00,1.037040e+00,2.341992e+00,9.438467e-01,1.896307e+00,6.363266e+00,7.002994e-02,-9.661942e-01,8.203936e-01,-8.949053e-01
std,NaN,NaN,2.476614e+02,NaN,NaN,NaN,2.002871e-01,7.584639e-01,1.570682e+00,1.010450e+00,9.640995e-01,2.163947e+00,8.836356e-01,1.647287e+00,NaN,2.529831e+02,NaN,NaN,NaN,2.095774e-01,8.028762e-01,1.518786e+00,1.026980e+00,9.374948e-01,2.112740e+00,8.952989e-01,1.626628e+00,NaN,1.964315e-01,1.627011e-01,8.177226e-01,8.067394e-01,1.896016e+00,7.597054e-01,1.463754e+00,1.549970e+00,1.445245e+00,1.052048e+00,1.441751e+00,1.099345e+00
min,NaN,NaN,-2.400000e+01,NaN,NaN,NaN,0.000000e+00,-3.141000e+00,-3.142000e+00,-1.343000e+00,-1.524000e+00,-4.760000e+00,-2.128000e+00,-3.242000e+00,NaN,-2.700000e+01,NaN,NaN,NaN,0.000000e+00,-3.141000e+00,-3.142000e+00,-1.343000e+00,-1.524000e+00,-4.760000e+00,-2.128000e+00,-3.242000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.571041e+00,-2.881375e+00,-2.165999e+00,-2.881375e+00,-2.165999e+00
25%,NaN,NaN,6.600000e+01,NaN,NaN,NaN,7.000000e-03,-1.863000e+00,-7.280000e-01,-1.006000e+00,-9.870000e-01,-1.505000e+00,-2.770000e-01,-1.262000e+00,NaN,9.700000e+01,NaN,NaN,NaN,9.000000e-03,-1.856000e+00,-7.130000e-01,-1.006000e+00,-9.870000e-01,-1.505000e+00,-2.770000e-01,-9.120000e-01,NaN,1.000000e+00,2.600000e-02,3.520000e-01,3.970000e-01,7.720000e-01,3.280000e-01,7.150000e-01,5.279086e+00,-1.106118e+00,-2.006822e+00,-1.356317e-01,-2.006822e+00
50%,NaN,NaN,1.430000e+02,NaN,NaN,NaN,8.500000e-02,-1.216000e+00,-2.930000e-01,-2.280000e-01,-2.790000e-01,5.330000e-01,3.930000e-01,-1.460000e-01,NaN,1.810000e+02,NaN,NaN,NaN,9.500000e-02,-1.246000e+00,-1.830000e-01,-2.280000e-01,-2.790000e-01,5.330000e-01,3.930000e-01,-1.460000e-01,NaN,1.000000e+00,1.070000e-01,7.910000e-01,8.850000e-01,2.063000e+00,8.020000e-01,1.674000e+00,5.983212e+00,4.948263e-01,-1.303724e+00,7.786310e-01,-1.255422e+00
75%,NaN,NaN,2.660000e+02,NaN,NaN,NaN,2.870000e-01,-1.077000e+00,2.302000e+00,9.450000e-01,4.650000e-01,1.502000e+00,1.045000e+00,1.313000e+00,NaN,3.070000e+02,NaN,NaN,NaN,3.030000e-01,-1.086000e+00,2.290000e+00,9.450000e-01,3.260000e-01,1.502000e+00,1.045000e+00,1.313000e+00,NaN,1.000000e+00,2.500000e-01,1.571000e+00,1.454000e+00,3.636000e+00,1.441000e+00,2.786000e+00,6.789931e+00,7.786310e-01,-3.599843e-01,2.149514e+00,-1.502629e-01


Double check feature types:

In [ ]:
print(combined_df.dtypes)

pdb_id               object
s_ch               category
s_resi                int64
s_ins              category
s_resn             category
s_ss8              category
s_rsa               float64
s_phi               float64
s_psi               float64
s_a1                float64
s_a2                float64
s_a3                float64
s_a4                float64
s_a5                float64
t_ch               category
t_resi                int64
t_ins              category
t_resn             category
t_ss8              category
t_rsa               float64
t_phi               float64
t_psi               float64
t_a1                float64
t_a2                float64
t_a3                float64
t_a4                float64
t_a5                float64
Interaction        category
same_chain            int64
delta_rsa           float64
delta_atchley_1     float64
delta_atchley_2     float64
delta_atchley_3     float64
delta_atchley_4     float64
delta_atchley_5     float64
ca_distance         

##  Missing/unclassified interactions
Unclassified interactions are removed.

In [ ]:
# Remove unclassified observations
combined_df = combined_df[combined_df['Interaction'] != 'MISSING']

# Remove unused category from the column
combined_df['Interaction'] = combined_df['Interaction'].cat.remove_unused_categories()

In [ ]:
print("Resulting dataset shape:", combined_df.shape)

Resulting dataset shape: (1802963, 40)


## Multiple interactions

 As we know, it is possible for residues to have more than one interaction type, thus the dataset might contain multiple observations referring to the same residue pair, each identifying a different interaction type between the two.

 To explore the entity of these multiple interactions we count the duplicate observations based on a subset of features that are sufficient to uniquely identify each single residues pair: the PDB ID, residues ID, residues insertion codes and chain IDs.

In [ ]:
id_cols = ['pdb_id', 's_resi', 't_resi', 's_ch', 't_ch', 's_ins', 't_ins']
combined_df[id_cols].drop_duplicates().shape

(1393382, 7)

In [ ]:
grouped_df = combined_df.groupby(id_cols, sort=False, observed=True)['Interaction'].apply(list).reset_index()

In [ ]:
grouped_df.head()

,pdb_id,s_resi,t_resi,s_ch,t_ch,s_ins,t_ins,Interaction
0,1b0y,28,31,A,A,,,[VDW]
1,1b0y,27,31,A,A,,,[HBOND]
2,1b0y,47,84,A,A,,,[HBOND]
3,1b0y,40,45,A,A,,,[HBOND]
4,1b0y,37,40,A,A,,,"[HBOND, VDW]"


In [ ]:
grouped_df.shape

(1393382, 8)

# Train/Test Split
A 80/20 training/test sets split is implemented. Subsequently, another 80/20 split is performed to exctract a validation set.

Each split is performed following two criteria:
1. Interactions in the same PDB structure (same *pdb_id*) are maintained together in the same set, preventing structures dispersion;
2. Each split is carried out stratifying the sets based on the interaction type labels, i.e. the original interaction proportions are maintained in all sets.

In [ ]:
# Create a summary DataFrame where each row is one pdb_id
# and is labeled with its dominant Interaction type (for stratification)
pdb_summary = combined_df.groupby('pdb_id')['Interaction'].agg(lambda x: x.mode()[0]).reset_index()

# Train-test split on pdb_ids, stratified by dominant interaction
train_pdbs, test_pdbs = train_test_split(
    pdb_summary['pdb_id'],
    test_size=0.2,
    random_state=42,
    stratify=pdb_summary['Interaction']
)

# Filter the original DataFrame based on selected pdb_ids
train_df_temp = combined_df[combined_df['pdb_id'].isin(train_pdbs)]  # temporary df
test_df = combined_df[combined_df['pdb_id'].isin(test_pdbs)]

In [ ]:
# Re-run on temporary training set summary DataFrame where each row is one pdb_id
# and is labeled with its dominant Interaction type (for stratification)
pdb_summary = train_df_temp.groupby('pdb_id')['Interaction'].agg(lambda x: x.mode()[0]).reset_index()

# Train-validation split on pdb_ids, stratified by dominant interaction
train_pdbs, val_pdbs = train_test_split(
    pdb_summary['pdb_id'],
    test_size=0.2,
    random_state=42,
    stratify=pdb_summary['Interaction']
)

# Filter the original DataFrame based on selected pdb_ids
train_df = train_df_temp[train_df_temp['pdb_id'].isin(train_pdbs)]
val_df = train_df_temp[train_df_temp['pdb_id'].isin(val_pdbs)]

The procedure ultimately yields a 64/16/20 training/validation/test split:

In [ ]:
print("Shape of Train dataset:", train_df.shape)
print("Shape of Test dataset:", test_df.shape)
print("Shape of Validation dataset:", val_df.shape)

Shape of Train dataset: (1159045, 40)
Shape of Test dataset: (351816, 40)
Shape of Validation dataset: (292102, 40)


In [ ]:
df_split = []
for df in [train_df, val_df, test_df]:
    df_split.append(round(df.shape[0] / combined_df.shape[0], 2))
print("Split sizes: {}".format(df_split))

Split sizes: [0.64, 0.16, 0.2]


Double check stratification:

In [ ]:
total_interaction_counts = train_df['Interaction'].value_counts() + test_df['Interaction'].value_counts() + val_df['Interaction'].value_counts()

train_interaction_counts = train_df['Interaction'].value_counts() / total_interaction_counts
print("Train Set -", train_interaction_counts, "\n")
val_interaction_counts = val_df['Interaction'].value_counts() / total_interaction_counts
print("Validation set -", val_interaction_counts, "\n")
test_interaction_counts = test_df['Interaction'].value_counts() / total_interaction_counts
print("Test set -", test_interaction_counts)

Train Set - Interaction
HBOND        0.643552
IONIC        0.651961
PICATION     0.634659
PIHBOND      0.650849
PIPISTACK    0.633239
SSBOND       0.676500
VDW          0.641907
Name: count, dtype: float64 

Validation set - Interaction
HBOND        0.161999
IONIC        0.159067
PICATION     0.159644
PIHBOND      0.144112
PIPISTACK    0.163563
SSBOND       0.177000
VDW          0.162121
Name: count, dtype: float64 

Test set - Interaction
HBOND        0.194450
IONIC        0.188972
PICATION     0.205697
PIHBOND      0.205038
PIPISTACK    0.203198
SSBOND       0.146500
VDW          0.195972
Name: count, dtype: float64


# Data augmentation and encoding


# Training set

Features and target are separated for next steps:

In [ ]:
# Separate features and target
X_train = train_df.drop(columns=['Interaction'])  # features
y_train = train_df['Interaction']  # target

## SMOTE NC



To augment data and mitigate underfitting of under-represented labels, we apply SMOTE NC, which is a version of SMOTE capable of handling properly categorical variables. We do so before OHE for two reasons:
1. Requres categorical variables, not OHE;
2. The dataset is smaller in size as no dummy variables are created yet.

It is applied on the training set only and augmented proportions are manually set based on the original frequencies of each class.

In [ ]:
interaction_counts_train = train_df['Interaction'].value_counts()
print(interaction_counts_train)

Interaction
HBOND        652035
VDW          453810
PIPISTACK     23326
IONIC         22063
PICATION       5347
SSBOND         1353
PIHBOND        1111
Name: count, dtype: int64


Based on these counts we can define manually a sampling strategy dictionary:

In [ ]:
sampling_strategy = {}

for label, count in interaction_counts_train.items():
    if count >= 100000:
        continue  # do not augment
    elif count >= 10000:
        sampling_strategy[label] = count * 5  # 5x for intermediate
    else:
        sampling_strategy[label] = count * 10  # 10x for rare

In [ ]:
# Separate the pdb_id from the features
pdb_ids = X_train['pdb_id'].reset_index(drop=True)
X = X_train.drop(columns=['pdb_id']).reset_index(drop=True)

# Retrieve updated categorical columns
categorical_cols = X.select_dtypes(include='category').columns.tolist()
categorical_indices = [X.columns.get_loc(col) for col in categorical_cols]

# Run SMOTE
smote_nc = SMOTENC(
    categorical_features=categorical_indices,
    sampling_strategy=sampling_strategy,
    random_state=42
)

# Resample training set
X_train_SMOTE, y_train_SMOTE = smote_nc.fit_resample(X.values, y_train.values)

# Determine how many new samples were generated
n_original = len(X)
n_resampled = len(X_train_SMOTE)
n_new = n_resampled - n_original

# Construct new pdb_id for synthetic samples
# For example, keep original pdb_ids, and generate synthetic pdb_ids like 'synthetic_0', 'synthetic_1', ...
new_pdb_ids = pd.Series([f"synthetic_{i}" for i in range(n_new)])

# Combine old and new pdb_ids
pdb_ids_resampled = pd.concat([pdb_ids, new_pdb_ids], ignore_index=True)

# Build the final dataframe
y_train_SMOTE = pd.Series(y_train_SMOTE, name='Interaction').astype('category')
X_train_SMOTE = pd.concat([pd.DataFrame(X_train_SMOTE, columns=X.columns), y_train_SMOTE], axis=1)
X_train_SMOTE['pdb_id'] = pdb_ids_resampled

# Reorder columns to have pdb_id first
cols = ['pdb_id'] + [col for col in X_train.columns if col not in ['pdb_id', 'Interaction']] + ['Interaction']
X_train_SMOTE = X_train_SMOTE[cols]

In [ ]:
X_train_SMOTE.tail()

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,same_chain,delta_rsa,delta_atchley_1,delta_atchley_2,delta_atchley_3,delta_atchley_4,delta_atchley_5,ca_distance,s_centroid_x,s_centroid_y,t_centroid_x,t_centroid_y,Interaction
1410895,synthetic_251850,A,44.0,,C,E,0.286285,-1.841577,2.246911,-1.343,0.465,-0.862,-1.02,-0.255,A,50.790291,,C,E,0.0,-2.168915,2.178674,-1.343,0.465,-0.862,-1.02,-0.255,1.0,0.286285,0.0,0.0,0.0,0.0,0.0,3.949307,-1.076392,-0.462708,2.492444,0.81338,SSBOND
1410896,synthetic_251851,A,1614.797371,,C,P,0.112415,-2.153707,0.488697,-1.343,0.465,-0.862,-1.02,-0.255,A,1628.168049,,C,E,0.216362,-2.016101,2.232445,-1.343,0.465,-0.862,-1.02,-0.255,1.0,0.103947,0.0,0.0,0.0,0.0,0.0,5.619668,-0.490302,-0.928267,1.024432,0.53436,SSBOND
1410897,synthetic_251852,X,173.0,,C,E,0.0,-2.028557,2.190991,-1.343,0.465,-0.862,-1.02,-0.255,L,180.0,,C,E,0.010308,-2.332413,2.726598,-1.343,0.465,-0.862,-1.02,-0.255,1.0,0.010308,0.0,0.0,0.0,0.0,0.0,3.866001,2.139425,0.048612,2.917936,1.143728,SSBOND
1410898,synthetic_251853,J,176.0,,C,E,0.0,-2.165016,2.76973,-1.343,0.465,-0.862,-1.02,-0.255,N,182.0,,C,E,0.0,-2.478185,2.637215,-1.343,0.465,-0.862,-1.02,-0.255,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.834432,2.139425,0.048612,2.917936,1.143728,SSBOND
1410899,synthetic_251854,A,31.951792,,C,E,0.146714,-2.745984,2.699681,-1.343,0.465,-0.862,-1.02,-0.255,A,73.048208,,C,P,0.158608,-1.447605,2.34985,-1.343,0.465,-0.862,-1.02,-0.255,1.0,0.097323,0.0,0.0,0.0,0.0,0.0,5.167603,-1.050515,-0.052663,0.634809,-0.080429,SSBOND


The augmented training set has size:

In [ ]:
print("Augmented set size:", X_train_SMOTE.shape)
print("Number of added observations:", abs(X_train_SMOTE.shape[0] - X_train.shape[0]))

Augmented set size: (1410900, 40)
Number of added observations: 251855


with the following labels frequencies:

In [ ]:
print(Counter(y_train_SMOTE))

Counter({'HBOND': 652035, 'VDW': 453810, 'PIPISTACK': 116630, 'IONIC': 110315, 'PICATION': 53470, 'SSBOND': 13530, 'PIHBOND': 11110})


## MultiLabel Binarizer

In the **Multiple interactions** section we explored the presence of residue pairs that map into multiple interactions of different type. In the current datasets, these pairs constitute an observation for each interaction type, effectively over-crowding the dataframe.

The MultiLabel Binarizer is used to handle this issue by merging such observations into a single one containing all the occurring interactions as dummy variables (0/1).

In [ ]:
feature_cols = [col for col in X_train_SMOTE.columns if col != 'Interaction']
grouped_train_df = X_train_SMOTE.groupby(feature_cols, sort=False, observed=True)['Interaction'].apply(set).reset_index()

# Reconstruct X and y
# The feature columns are all columns except the last one ('Interaction') after reset_index
X_train_MLB = grouped_train_df.iloc[:, :-1]
y_train_grouped = grouped_train_df.iloc[:, -1].tolist()

# Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_train_MLB = mlb.fit_transform(y_train_grouped)

In [ ]:
grouped_train_df.shape

(1147498, 40)

In [ ]:
grouped_train_df.head()

,pdb_id,s_ch,s_resi,s_ins,s_resn,s_ss8,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,t_ch,t_resi,t_ins,t_resn,t_ss8,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,same_chain,delta_rsa,delta_atchley_1,delta_atchley_2,delta_atchley_3,delta_atchley_4,delta_atchley_5,ca_distance,s_centroid_x,s_centroid_y,t_centroid_x,t_centroid_y,Interaction
0,1bs9,A,121.0,,A,H,0.000,-1.203,-0.763,-0.591,-1.302,-0.733,1.570,-0.146,A,124.0,,M,H,0.128,-1.560,-0.151,-0.663,-1.524,2.219,-1.005,1.212,1.0,0.128,0.072,0.222,2.952,2.575,1.358,5.185897,2.305979,-1.498816,2.149514,-0.802992,{HBOND}
1,1bs9,A,201.0,,F,H,0.142,-1.109,-0.835,-1.006,-0.590,1.891,-0.397,0.412,A,204.0,,S,H,0.646,-1.137,-0.508,-0.228,1.399,-4.760,0.670,-2.647,1.0,0.504,0.778,1.989,6.651,1.067,3.059,5.103776,2.305979,-1.498816,0.778631,-2.165999,"{VDW, HBOND}"
2,1bs9,A,190.0,,Y,H,0.000,-1.222,-0.397,0.260,0.830,3.097,-0.838,1.512,A,194.0,,Y,H,0.081,-2.080,0.042,0.260,0.830,3.097,-0.838,1.512,1.0,0.081,0.000,0.000,0.000,0.000,0.000,5.500035,-0.135632,-1.891373,-0.135632,-1.891373,"{PIPISTACK, HBOND}"
3,1bs9,A,94.0,,E,H,0.000,-0.986,-0.806,1.357,-1.453,1.477,0.113,-0.837,A,105.0,,D,B,0.012,-2.537,1.320,1.050,0.302,-3.656,-0.259,-3.242,1.0,0.012,0.307,1.755,5.133,0.372,2.405,8.813940,-1.106118,-1.339661,-1.106118,-1.339661,"{VDW, HBOND}"
4,1bs9,A,69.0,,A,H,0.066,-1.051,-0.818,-0.591,-1.302,-0.733,1.570,-0.146,A,73.0,,N,H,0.248,-1.136,-0.707,0.945,0.828,1.299,-0.169,0.933,1.0,0.182,1.536,2.130,2.032,1.739,1.079,5.717558,-1.106118,-1.339661,-1.106118,-1.339661,{HBOND}


In [ ]:
print("Observations merged: ", X_train_SMOTE.shape[0] - X_train_MLB.shape[0])

Observations merged:  263402


Column names are assigned to the MLB numpy array output:

In [ ]:
# Assign column names to target
y_train_final = pd.DataFrame(y_train_MLB, columns=mlb.classes_)
y_train_final.head()

,HBOND,IONIC,PICATION,PIHBOND,PIPISTACK,SSBOND,VDW
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,1
2,1,0,0,0,1,0,0
3,1,0,0,0,0,0,1
4,1,0,0,0,0,0,0


In [ ]:
X_train_MLB.drop(columns=['pdb_id', 's_ch', 't_ch', 's_ins', 't_ins'], inplace=True)  # s_resi, t_resi

In [ ]:
# Restore categorical features after dropping some columns
categorical_cols = [
    's_resn', 't_resn', 's_ss8', 't_ss8'
]

## One-Hot Encoding

We proceed with the One-Hot Encoding (OHE) of the categorical variables in the dataset.

In [ ]:
# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit training
X_train_OHE = encoder.fit_transform(X_train_MLB[categorical_cols])

# Drop original and append encoded features
X_train_final = X_train_MLB.drop(columns=categorical_cols).reset_index(drop=True)

# Get new column names
encoded_cols = encoder.get_feature_names_out(categorical_cols)

X_train_final = pd.concat([
    X_train_final,
    pd.DataFrame(X_train_OHE, columns=encoded_cols)
], axis=1)

In [ ]:
joblib.dump(encoder, f'{folder_path}/Structural Bioinfo PROJECT/onehot_encoder.pkl')

['/content/drive/MyDrive/Structural Bioinfo PROJECT/onehot_encoder.pkl']

In [ ]:
print("Shape of Train dataset:", X_train_final.shape)
X_train_final.head()

Shape of Train dataset: (1147498, 88)


,s_resi,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,t_resi,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,same_chain,delta_rsa,delta_atchley_1,delta_atchley_2,delta_atchley_3,delta_atchley_4,delta_atchley_5,ca_distance,s_centroid_x,s_centroid_y,t_centroid_x,t_centroid_y,s_resn_A,s_resn_C,s_resn_D,s_resn_E,s_resn_F,s_resn_G,s_resn_H,s_resn_I,s_resn_K,s_resn_L,s_resn_M,s_resn_N,s_resn_P,s_resn_Q,s_resn_R,s_resn_S,s_resn_T,s_resn_V,s_resn_W,s_resn_Y,t_resn_A,t_resn_C,t_resn_D,t_resn_E,t_resn_F,t_resn_G,t_resn_H,t_resn_I,t_resn_K,t_resn_L,t_resn_M,t_resn_N,t_resn_P,t_resn_Q,t_resn_R,t_resn_S,t_resn_T,t_resn_V,t_resn_W,t_resn_Y,s_ss8_-,s_ss8_B,s_ss8_E,s_ss8_G,s_ss8_H,s_ss8_I,s_ss8_P,s_ss8_S,s_ss8_T,t_ss8_-,t_ss8_B,t_ss8_E,t_ss8_G,t_ss8_H,t_ss8_I,t_ss8_P,t_ss8_S,t_ss8_T
0,121.0,0.000,-1.203,-0.763,-0.591,-1.302,-0.733,1.570,-0.146,124.0,0.128,-1.560,-0.151,-0.663,-1.524,2.219,-1.005,1.212,1.0,0.128,0.072,0.222,2.952,2.575,1.358,5.185897,2.305979,-1.498816,2.149514,-0.802992,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,201.0,0.142,-1.109,-0.835,-1.006,-0.590,1.891,-0.397,0.412,204.0,0.646,-1.137,-0.508,-0.228,1.399,-4.760,0.670,-2.647,1.0,0.504,0.778,1.989,6.651,1.067,3.059,5.103776,2.305979,-1.498816,0.778631,-2.165999,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,190.0,0.000,-1.222,-0.397,0.260,0.830,3.097,-0.838,1.512,194.0,0.081,-2.080,0.042,0.260,0.830,3.097,-0.838,1.512,1.0,0.081,0.000,0.000,0.000,0.000,0.000,5.500035,-0.135632,-1.891373,-0.135632,-1.891373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,94.0,0.000,-0.986,-0.806,1.357,-1.453,1.477,0.113,-0.837,105.0,0.012,-2.537,1.320,1.050,0.302,-3.656,-0.259,-3.242,1.0,0.012,0.307,1.755,5.133,0.372,2.405,8.813940,-1.106118,-1.339661,-1.106118,-1.339661,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,69.0,0.066,-1.051,-0.818,-0.591,-1.302,-0.733,1.570,-0.146,73.0,0.248,-1.136,-0.707,0.945,0.828,1.299,-0.169,0.933,1.0,0.182,1.536,2.130,2.032,1.739,1.079,5.717558,-1.106118,-1.339661,-1.106118,-1.339661,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Resulting dataframe statistics are assesed:

In [ ]:
X_train_final.describe(include='all')

,s_resi,s_rsa,s_phi,s_psi,s_a1,s_a2,s_a3,s_a4,s_a5,t_resi,t_rsa,t_phi,t_psi,t_a1,t_a2,t_a3,t_a4,t_a5,same_chain,delta_rsa,delta_atchley_1,delta_atchley_2,delta_atchley_3,delta_atchley_4,delta_atchley_5,ca_distance,s_centroid_x,s_centroid_y,t_centroid_x,t_centroid_y,s_resn_A,s_resn_C,s_resn_D,s_resn_E,s_resn_F,s_resn_G,s_resn_H,s_resn_I,s_resn_K,s_resn_L,s_resn_M,s_resn_N,s_resn_P,s_resn_Q,s_resn_R,s_resn_S,s_resn_T,s_resn_V,s_resn_W,s_resn_Y,t_resn_A,t_resn_C,t_resn_D,t_resn_E,t_resn_F,t_resn_G,t_resn_H,t_resn_I,t_resn_K,t_resn_L,t_resn_M,t_resn_N,t_resn_P,t_resn_Q,t_resn_R,t_resn_S,t_resn_T,t_resn_V,t_resn_W,t_resn_Y,s_ss8_-,s_ss8_B,s_ss8_E,s_ss8_G,s_ss8_H,s_ss8_I,s_ss8_P,s_ss8_S,s_ss8_T,t_ss8_-,t_ss8_B,t_ss8_E,t_ss8_G,t_ss8_H,t_ss8_I,t_ss8_P,t_ss8_S,t_ss8_T
count,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06,1.147498e+06
mean,1.981975e+02,1.711245e-01,-1.394678e+00,5.722613e-01,1.324133e-01,-1.035014e-01,9.314794e-02,8.973232e-02,5.455873e-02,2.343407e+02,1.852072e-01,-1.378113e+00,5.601196e-01,1.550668e-01,-1.440923e-01,1.772425e-01,8.539873e-02,1.698009e-01,9.571274e-01,1.522490e-01,9.773058e-01,9.549016e-01,2.223847e+00,8.877055e-01,1.959536e+00,6.608649e+00,1.256237e-01,-9.965846e-01,7.852770e-01,-9.362168e-01,5.941013e-02,2.217956e-02,7.144326e-02,7.397311e-02,7.333085e-02,4.043319e-02,4.249419e-02,4.327502e-02,6.451427e-02,6.823716e-02,1.587802e-02,3.852556e-02,2.339873e-02,3.118960e-02,8.358359e-02,4.592339e-02,4.185454e-02,5.074954e-02,3.844625e-02,7.116004e-02,6.324107e-02,2.171943e-02,6.010729e-02,7.274871e-02,7.174653e-02,4.560444e-02,4.565672e-02,4.173776e-02,7.181973e-02,6.969250e-02,1.700918e-02,3.794691e-02,1.052028e-02,3.545627e-02,9.579537e-02,4.078351e-02,3.905715e-02,4.862841e-02,3.921575e-02,7.151298e-02,1.462887e-01,1.115296e-02,2.345965e-01,3.374821e-02,4.370971e-01,6.832256e-03,1.409501e-02,5.890293e-02,5.728637e-02,1.245850e-01,1.039217e-02,2.305468e-01,3.581096e-02,4.297001e-01,8.243152e-03,1.188760e-02,6.343715e-02,8.539710e-02
std,2.211298e+02,1.917489e-01,7.225683e-01,1.529876e+00,1.026830e+00,8.949891e-01,2.133928e+00,9.127528e-01,1.657422e+00,2.260629e+02,2.020374e-01,7.672229e-01,1.471443e+00,1.042264e+00,8.649798e-01,2.074001e+00,9.248208e-01,1.640239e+00,1.975345e-01,1.506930e-01,7.997462e-01,7.533611e-01,1.868336e+00,7.391516e-01,1.550439e+00,1.694625e+00,1.380263e+00,1.023583e+00,1.394480e+00,1.063133e+00,2.363908e-01,1.472673e-01,2.575639e-01,2.617272e-01,2.606789e-01,1.969730e-01,2.017138e-01,2.034756e-01,2.456669e-01,2.521525e-01,1.250037e-01,1.924613e-01,1.511663e-01,1.738299e-01,2.767624e-01,2.093191e-01,2.002568e-01,2.194859e-01,1.922711e-01,2.570921e-01,2.433962e-01,1.457660e-01,2.376856e-01,2.597237e-01,2.580679e-01,2.086258e-01,2.087396e-01,1.999894e-01,2.581893e-01,2.546282e-01,1.293054e-01,1.910680e-01,1.020275e-01,1.849301e-01,2.943105e-01,1.977884e-01,1.937311e-01,2.150900e-01,1.941080e-01,2.576799e-01,3.533956e-0

## Saving

In [ ]:
save_df_as_tsv(pd.concat([X_train_final, y_train_final], axis=1), tsv_path, 'train_set_new_smote')

Saved TSV to: /content/drive/MyDrive/Structural Bioinfo PROJECT/datasets/train_set_new_smote.tsv


# Test set

## MultiLabel Binarizer

In [ ]:
grouped_test_df = test_df.groupby(feature_cols, sort=False, observed=True)['Interaction'].apply(set).reset_index()

# Reconstruct X and y
# The feature columns are all columns except the last one ('Interaction') after reset_index
X_test_MLB = grouped_test_df.iloc[:, :-1]
y_test_grouped = grouped_test_df.iloc[:, -1].tolist()

# Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_test_MLB = mlb.fit_transform(y_test_grouped)

In [ ]:
test_df.shape

(351816, 40)

In [ ]:
grouped_test_df.shape

(272070, 40)

In [ ]:
print("Observations merged: ", test_df.shape[0] - grouped_test_df.shape[0])

Observations merged:  79746


In [ ]:
# Assign column names to target
y_test_final = pd.DataFrame(y_test_MLB, columns=mlb.classes_)
y_test_final.head()

,HBOND,IONIC,PICATION,PIHBOND,PIPISTACK,SSBOND,VDW
0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,1


In [ ]:
X_test_MLB.drop(columns=['pdb_id', 's_ch', 't_ch', 's_ins', 't_ins'], inplace=True)

## One-Hot Encoding

In [ ]:
# Fit only on training
X_test_OHE = encoder.transform(X_test_MLB[categorical_cols])

# Drop original and append encoded features
X_test_final = X_test_MLB.drop(columns=categorical_cols).reset_index(drop=True)

X_test_final = pd.concat([
    X_test_final,
    pd.DataFrame(X_test_OHE, columns=encoded_cols)
], axis=1)

In [ ]:
print("Shape of Test dataset:", X_test_final.shape)

Shape of Test dataset: (272070, 88)


## Saving

In [ ]:
save_df_as_tsv(pd.concat([X_test_final, y_test_final], axis=1), tsv_path, 'test_set_new')


Saved TSV to: /content/drive/MyDrive/Structural Bioinfo PROJECT/datasets/test_set_new.tsv


# Validation set

## MultiLabel Binarizer

In [ ]:
grouped_val_df = val_df.groupby(feature_cols, sort=False, observed=True)['Interaction'].apply(set).reset_index()

# Reconstruct X and y
# The feature columns are all columns except the last one ('Interaction') after reset_index
X_val_MLB = grouped_val_df.iloc[:, :-1]
y_val_grouped = grouped_val_df.iloc[:, -1].tolist()

# Apply MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_val_MLB = mlb.fit_transform(y_val_grouped)

In [ ]:
val_df.shape

(292102, 40)

In [ ]:
grouped_val_df.shape

(225669, 40)

In [ ]:
print("Observations merged: ", val_df.shape[0] - grouped_val_df.shape[0])

Observations merged:  66433


In [ ]:
# Assign column names to target
y_val_final = pd.DataFrame(y_val_MLB, columns=mlb.classes_)
y_val_final.head()

,HBOND,IONIC,PICATION,PIHBOND,PIPISTACK,SSBOND,VDW
0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0


In [ ]:
X_val_MLB.drop(columns=['pdb_id', 's_ch', 't_ch', 's_ins', 't_ins'], inplace=True)

## One-Hot Encoding

In [ ]:
# Fit only on training
X_val_OHE = encoder.transform(X_val_MLB[categorical_cols])

# Drop original and append encoded features
X_val_final = X_val_MLB.drop(columns=categorical_cols).reset_index(drop=True)

X_val_final = pd.concat([
    X_val_final,
    pd.DataFrame(X_val_OHE, columns=encoded_cols)
], axis=1)

In [ ]:
print("Shape of Validation dataset:", X_val_final.shape)

Shape of Validation dataset: (225669, 88)


## Saving

In [ ]:
save_df_as_tsv(pd.concat([X_val_final, y_val_final], axis=1), tsv_path, 'val_set_new')


Saved TSV to: /content/drive/MyDrive/Structural Bioinfo PROJECT/datasets/val_set_new.tsv
